In [1]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 721.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 9.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.9/408.9 kB 20.4 MB/s eta 0:00:00


In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import timm  # Make sure timm is installed: pip install timm
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =======================
# 1. Data Preparation
# =======================
# CIFAR-10 dataset; note that we resize images to 224x224 because most pre-trained models (e.g. DINOv2) expect larger inputs.
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Using ImageNet normalization
                         std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [19]:
# =======================
# 2. Model Setup
# =======================
feature_extractor = torch.hub.load('facebookresearch/dinov2', "dinov2_vitb14")
# No need to call reset_classifier; the head is already Identity.
feature_extractor.eval()

# Freeze the backbone parameters
for param in feature_extractor.parameters():
    param.requires_grad = False

# Determine the feature dimension (e.g. 768 for ViT-B/14)
num_features = 768
num_classes = 10  # CIFAR-10 has 10 classes

# Create a linear classifier on top of the frozen features.
linear_classifier = nn.Linear(num_features, num_classes)
linear_classifier.to(device)


Using cache found in /Users/rkovalch/.cache/torch/hub/facebookresearch_dinov2_main


Linear(in_features=768, out_features=10, bias=True)

In [20]:
# =======================
# 3. Training Setup
# =======================
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(linear_classifier.parameters(), lr=0.01, momentum=0.9)
num_epochs = 10


In [25]:
# =======================
# 4. Training Loop (Linear Probe)
# =======================
from tqdm import tqdm

for epoch in tqdm(range(num_epochs), desc="epochs progress"):
    linear_classifier.train()
    running_loss = 0.0
    total = 0
    correct = 0

    for inputs, labels in tqdm(train_loader, desc="train loader"):
        inputs, labels = inputs.to(device), labels.to(device)

        # Extract features using the frozen DINOv2 backbone.
        with torch.no_grad():
            features_dict = feature_extractor.forward_features(inputs)
            # Extract the class token from the returned dictionary.
            features = features_dict["x_norm_clstoken"]
            # Optionally, if your model returns a 4D tensor and you want to average pool patch tokens,
            # you could modify accordingly, e.g.:
            # features = features_dict["x_norm_patchtokens"]
            # features = features.mean(dim=[2, 3])

        # Forward pass through the linear classifier.
        outputs = linear_classifier(features)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total * 100
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")


train loader:  11%|█         | 84/782 [05:23<47:25,  4.08s/it]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x117392950>
Traceback (most recent call last):
  File "/Users/rkovalch/miniconda3/envs/cv-env/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/Users/rkovalch/miniconda3/envs/cv-env/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1443, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/Users/rkovalch/miniconda3/envs/cv-env/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/Users/rkovalch/miniconda3/envs/cv-env/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/Users/rkovalch/miniconda3/envs/cv-env/lib/python3.10/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/Users/

KeyboardInterrupt: 